# Hybrid Search using RRF

In this example we'll use the *reciprocal rank fusion algorithm* to combine the results of **BM25** and **kNN semantic search**. We'll use the same dataset we used in our quickstart guide.

You can use RRF for hybrid search out of the box, without any additional configuration. This example demonstrates how RRF ranking works at a basic level.

## Initialize the Elasticsearch client

Now, we need to initialize the Elasticsearch client. We will use the [Elasticsearch client for .NET](https://www.elastic.co/guide/en/elasticsearch/client/net-api/current/index.html) to connect to Elasticsearch.


#### Install packages and import modules

In [ ]:
#r "nuget: Elastic.Clients.Elasticsearch"

#!import ./Utils.cs

#!set --name elasticCloudId --value @input
#!set --name elasticCloudKey --value @input

#### Initialize Client

In [ ]:
using Elastic.Transport;
using Elastic.Clients.Elasticsearch;
using Elastic.Transport.Products.Elasticsearch;

var elasticSettings = new ElasticsearchClientSettings(
    elasticCloudId, new ApiKey(elasticCloudKey))
    .DisableDirectStreaming()
    .ServerCertificateValidationCallback(CertificateValidations.AllowAll);

var client = new ElasticsearchClient(elasticSettings);

#### Test the Client

Before you continue, confirm that the client has connected with this test.


In [ ]:
var info = await client.InfoAsync();

DumpResponse(info);

## Define Model


In [ ]:
using System.Text.Json.Serialization;

public class Book
{
    [JsonPropertyName("title")]
    public string Title { get; set; }

    [JsonPropertyName("summary")]
    public string Summary { get; set; }

    [JsonPropertyName("authors")]
    public List<string> Authors { get; set; }

    [JsonPropertyName("publish_date")]
    public DateTime publish_date { get; set; }

    [JsonPropertyName("num_reviews")]
    public int num_reviews { get; set; }

    [JsonPropertyName("publisher")]
    public string Publisher { get; set; }


    public float[] TitleVector { get; set; }
}

#### Pretty printing Elasticsearch responses

Let's add a helper function to print Elasticsearch responses in a readable format.

In [ ]:
void PrettyPrint(SearchResponse<Book> searchResponse) => searchResponse.Hits
    .Select(x => new { 
        Title = x.Source.Title,
        Score = x.Score,
        Rank = x.Rank,
        Summary = x.Source.Summary,
    })
    .DisplayTable();

### Setup the Embedding Model


In [ ]:
#r "nuget: Microsoft.Extensions.AI.OpenAI, 9.0.0-preview.*"
#r "nuget: Azure.AI.OpenAI, 2.0.0"

In [ ]:
using Azure.AI.OpenAI;
using Microsoft.Extensions.AI;

AzureOpenAIClient aiClient = new AzureOpenAIClient(
    new Uri(envs["AZURE_OPENAI_ENDPOINT"]),
    new System.ClientModel.ApiKeyCredential(envs["AZURE_OPENAI_APIKEY"]));

IEmbeddingGenerator<string,Embedding<float>> generator = aiClient
    .AsEmbeddingGenerator(modelId: "text-embedding-3-small");

var textEmeddingDimension = 384;

async Task<float[]> ToEmbedding(string text) {
    GeneratedEmbeddings<Embedding<float>> embeddings = await generator
        .GenerateAsync([text], new EmbeddingGenerationOptions{
            Dimensions = textEmeddingDimension
        });

    return embeddings.First().Vector.ToArray();
}

## Querying Documents with Hybrid Search

🔐 NOTE: to run the queries that follow you need the book_index dataset from our [00-quick-start.ipynb](./00-quick-start.ipynb). If you haven't worked through the quick start, please follow the steps described there to create an Elasticsearch deployment with the dataset in it, and then come back to run the queries here.

Now we need to perform a query using two different search strategies:

* Semantic search using the "all-MiniLM-L6-v2" embedding model
* Keyword search using the "title" field

We then use *Reciprocal Rank Fusion (RRF)* to balance the scores to provide a final list of documents, ranked in order of relevance. RRF is a ranking algorithm for combining results from different information retrieval strategies.

Note tha we use `_rank` to show our top-ranked documents.

In [ ]:
var searchQuery = "python programming";
var queryEmbedding = await ToEmbedding(searchQuery);

var searchResponse = await client.SearchAsync<Book>(s => s
    .Index("book_index")
    .Query(d => d.Match(m => m.Field(f => f.Summary).Query(searchQuery)))
    .Knn(d => d
        .Field(f => f.TitleVector)
        .QueryVector(queryEmbedding)
        .k(5)
        .NumCandidates(10))
    .Rank(r => r.Rrf(rrf => {}))
);

// DumpRequest(searchResponse);
PrettyPrint(searchResponse);


Title,Score,Rank,Summary
Python Crash Course,0.032786883,1,"A fast-paced, no-nonsense guide to programming in Python"
The Pragmatic Programmer: Your Journey to Mastery,0.03175403,2,A guide to pragmatic programming for software engineers and developers
Eloquent JavaScript,0.016129032,3,A modern introduction to programming
You Don't Know JS: Up & Going,0.015873017,4,Introduction to JavaScript and programming as a whole
JavaScript: The Good Parts,0.015873017,5,A deep dive into the parts of JavaScript that are essential to writing maintainable code
The Clean Coder: A Code of Conduct for Professional Programmers,0.015625,6,A guide to professional conduct in the field of software engineering
Design Patterns: Elements of Reusable Object-Oriented Software,0.015384615,7,Guide to design patterns that can be used in any object-oriented language
